## Prueba de modelos ##

### Version 4: Modelo con 2 DER, planificación anual, granularidad horaria ###

Si el modelo tiene que optimizar el balance por horas, tiene 8640 VD, pero sigue teniendo 2 VD binarias

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import pandapower as pp

In [ ]:
import pandas as pd

In [ ]:
import pyomo.environ as pe

In [ ]:
import math

In [ ]:
import random

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [1]:
import mgmo as mo

In [2]:
mo.ScenesMega

AttributeError: module 'mgmo' has no attribute 'ScenesMega'

In [ ]:
net = pp.create_empty_network() 
b0 = pp.create_bus(net, vn_kv=13.2)

pp.create_ext_grid(net, bus=b0)

#ramal 1
b11 = pp.create_bus(net, vn_kv=13.2)
b12 = pp.create_bus(net, vn_kv=13.2)

pp.create_line(net, from_bus=b0, to_bus=b11, length_km=0.8, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b11, to_bus=b12, length_km=2.1, std_type="NAYY 4x50 SE")

pp.create_sgen(net, b11, p_mw=0.075, q_mvar=0.025, max_p_mw=0.15, max_q_mvar=0.15)
pp.create_load(net, bus=b12, p_mw=0.350)

#ramal 2
b21 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b21, length_km=5.0, std_type="NAYY 4x50 SE")
pp.create_sgen(net, b21, p_mw=0.15, q_mvar=0.00, max_p_mw=0.40, max_q_mvar=0.10)
##pp.create_storage(net, bus=b21, p_mw=0.25, q_mvar=0.1, max_e_mwh = 3.0)

#ramal 3
b31 = pp.create_bus(net, vn_kv=13.2)
b32 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b31, length_km=0.5, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b31, to_bus=b32, length_km=1.5, std_type="NAYY 4x50 SE")
pp.create_sgen(net, b31, p_mw=0.08, q_mvar=0.00, max_p_mw=0.30, max_q_mvar=0.10)
pp.create_load(net, bus=b32, p_mw=0.85, q_mvar=0.33)


#ramal 4
b41 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b41, length_km=0.5, std_type="NAYY 4x50 SE")
pp.create_load(net, bus=b41, p_mw=0.22, q_mvar=0.05)
##pp.create_storage(net, bus=b41, p_mw=0.5, q_mvar=0.1, max_e_mwh = 5.0)



In [ ]:
 pp.runpp(net)

In [ ]:
#print(net.res_bus.vm_pu)
#print(net.res_line)

In [ ]:
print(net)

In [ ]:
net.sgen

In [ ]:
net.storage

Agregamos los parámetros del modelo a los elementos de la red.

In [ ]:
mo.network_precondition(net)

Vemos como queda por ejemplo el elemento de red externa:

In [ ]:
net.ext_grid

Defino una función normal de Python para que nos va a dar la componente lineal del costo de la energía

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

In [ ]:
net.ext_grid['model'][0] = mo.Resources.ExtGrid('SET', pr_mw = 5.0, pa_pu = 1.0, 
                                                oc_1_mu = mo.Simulation.ElectricityCostSimulator)
net.ext_grid['name'][0] = 'SET'


In [ ]:
net.ext_grid

In [ ]:
net.load['model'][0] = mo.Resources.Load('Tornquinst', pr_mw = net.load['p_mw'][0], pa_pu = mo.Simulation.DemandSimulator)

net.load['model'][1] = mo.Resources.Load('Bell Ville', pr_mw = net.load['p_mw'][1], pa_pu = mo.Simulation.DemandSimulator)

net.load['model'][2] = mo.Resources.Load('City Bell', pr_mw = net.load['p_mw'][2], pa_pu = mo.Simulation.DemandSimulator)

In [ ]:
net.load

In [ ]:
m = mo.Resources.PVGenerator('PV1',  ic_0_mu = 10.0e3, ic_1_mu = 1.35e6, oc_0_mu = 15.0e3/8760, oc_1_mu = 0.0)
net.sgen['name'][0] = m.name
net.sgen['model'][0] = m

m = mo.Resources.PVGenerator('PV2',  ic_0_mu = 10.0e3, ic_1_mu = 1.45e6, oc_0_mu = 14.0e3/8760, oc_1_mu = 0.0)
net.sgen['name'][2] = m.name
net.sgen['model'][2] = m

In [ ]:
net.sgen['model'][1] = mo.Resources.WTGeneratorDiscrete('WT 1', ic_0_mu = 10.0e3, unit_size_mw = 0.800, unit_cost_mu = 1200e3, oc_0_mu = 0.00,
                                                     oc_1_mu = 15.0, cut_in = 3.0, v_rated = 12.0, cut_out = 34.0)   #Enercom E-53 800 kW

In [ ]:
net.sgen

## Scenes construction

### Climatic Data

In [ ]:
Mean_Monthly_Solar_Irradiance = [7.5e3, 7.2e3, 6.5e3, 5.5e3, 4.5e3, 3.5e3, 2.5e3, 3.5e3, 4.5e3, 6.0e3, 7.0e3, 8.5e3]

In [ ]:
sun_synth = mo.Simulation.MonthlySolarIrradianceSimulator(latitude = -40.5490, longitude = -62.2438, 
                                                       monthly_average_irradiation = Mean_Monthly_Solar_Irradiance)

In [ ]:
wind_synth = mo.Simulation.Weibull(c = 7.5 / 0.886, k = 2)

In [ ]:
s = mo.Scenes.SceneBuilder(years = 20, subperiods = 6, days_in_subperiods = 7, dt = 1, growth_rate = 0.025, discount_rate = 0.10)
s.additional_columns['solar_irradiance'] = sun_synth
s.additional_columns['wind_speed'] = wind_synth
scenes = s.build_scenes()
scenes

In [ ]:
mo.Scenes

In [ ]:
mo.Test

In [ ]:
dias = 365
años = 5

In [ ]:
#el rango de horas:
T_i = range(24)

#el rango de dias:
Dias_i = range(dias)

Escenarios_i = range(len(T_i)*len(Dias_i))

In [ ]:
m_s_base = [{'y': 0, 'd': 0, 'dd':1.0, 'h': 0.0, 'dt': 1.0, 'temp': 20.0, 'I':1000.0, 'wv': 10.0, 'eg': 1.0}]

registros = len(Dias_i)*len(T_i)

Escenarios =  pd.DataFrame(m_s_base, index = Escenarios_i)

reg = 0
for d in Dias_i:
    for h in T_i:
        Escenarios['d'][reg] = d
        Escenarios['h'][reg] = h
        Escenarios['I'][reg] = Solar.solar_irradiance_seasoned_randomized(d, h)
        Escenarios['wv'][reg] = random.uniform(1.0, 20.0)
        reg += 1

In [ ]:
Escenarios

## Construcción del Modelo

Construccion automatica del modelo
Es un modelo de dimensionamiento de 3 DERs (WT+PV)

In [ ]:
model_writer = mo.SimpleModelWriter(net, Escenarios)

In [ ]:
model = model_writer.create_model()

In [ ]:
##model.pprint()

In [ ]:
#solver_manager = pe.SolverManagerFactory('neos')
#result_obj = solver_manager.solve(model.model, opt='cplex')

opt = pe.SolverFactory('cplex')
#opt = pe.SolverFactory('glpk')
result_obj= opt.solve(model, tee=True)

In [ ]:
getattr(model_writer.model, 'PV1_create').value

In [ ]:
model_writer.backconfigure_network()

In [ ]:
model_writer.net.sgen

In [ ]:
model_writer.net.storage

In [ ]:
Resultados = model_writer.get_scenes_results()

In [ ]:
Resultados

In [ ]:
# Set the plot curve with markers and a title
fig, ax = plt.subplots(1, 1)

fig.set_size_inches(10,6)
a=180*24
#b=360
b=180*24+5*24
for col in Resultados:
    ax.plot(Resultados[col][a:b], label = col)

#ax.plot(Resultados['Tornquinst'][a:b]+Resultados['Bell Ville'][a:b]+Resultados['City Bell'][a:b], label = 'Load')

#ax.plot(Resultados['time'][240:360], Resultados['p_mw_PV'][240:360], label = 'PV')
#ax.plot(Resultados['time'][240:360], Resultados['p_mw_Load'][240:360], label = 'Load')

ax.set_xlabel('time')
ax.set_ylabel('P[mw]')

ax.legend()

ax.grid(True)


plt.show()

In [ ]:
df = pd.DataFrame([], index = range(100))

In [ ]:
df['Col 1'] = [i for i in range(100)]

In [ ]:
df

In [ ]:
for c in Resultados:
    print(Resultados[c][10])

In [ ]:
len(Resultados)